# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [1]:
# 필요한 라이브러리를 설치합니다.
%pip install timm

Note: you may need to restart the kernel to use updated packages.


## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [20]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [4]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [5]:
# 학습을 위한 함수입니다.
def train_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [6]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '/home/data'

# model config
model_name = 'efficientnet_b2' # 'resnet50' 'efficientnet-b0', ...

# 하이퍼파라미터를 설정합니다.
img_size = 224
#의미: 입력 이미지의 크기를 지정합니다. 모델에 입력하기 전에 이미지를 이 크기로 조정합니다.
#변화:
#크기를 늘리면 모델이 더 많은 세부 정보를 학습할 수 있지만, 계산 비용이 증가합니다.
#크기를 줄이면 계산 비용은 감소하지만, 정보 손실로 인해 성능이 떨어질 수 있습니다.
LR = 1e-4
#의미: 학습률을 지정합니다. 모델 파라미터를 업데이트할 때 사용되는 비율입니다.
#변화:
#학습률이 높으면 학습이 빠르게 진행되지만, 최적값을 놓치거나 불안정할 수 있습니다.
#학습률이 낮으면 안정적으로 학습되지만, 시간이 오래 걸릴 수 있습니다.
EPOCHS = 10
#의미: 전체 데이터셋을 학습할 횟수를 지정합니다.
#변화:
#에포크 수가 많으면 모델이 데이터에 대해 더 잘 학습할 수 있지만, 과적합(overfitting)의 위험이 있습니다.
#에포크 수가 적으면 과소적합(underfitting)이 발생할 수 있습니다.
BATCH_SIZE = 64
#의미: 한 번에 학습할 데이터 샘플의 수를 지정합니다.
#변화:
#배치 크기가 크면 학습이 안정적이고, GPU의 효율성을 높일 수 있습니다. 하지만 메모리 사용량이 증가합니다.
#배치 크기가 작으면 메모리 사용량이 적고, 미세한 업데이트가 가능하지만, 학습이 불안정할 수 있습니다.
num_workers = 4
#의미: 데이터 로딩을 위해 사용할 병렬 작업자의 수를 지정합니다.
#변화:
#작업자 수가 많으면 데이터 로딩 속도가 빨라져 학습이 더 원활하게 진행될 수 있습니다.
#너무 많은 작업자를 사용하면 시스템의 다른 작업과 충돌하거나 메모리 문제가 발생할 수 있습니다.

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [7]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [9]:
# Dataset 정의
trn_dataset = ImageDataset(
    # "/home/data/train.csv",
    "/home/data/train_augmented_30D.csv",
    # "/home/data/train",
    # "/home/data/rotate_train",
    "/home/data/augmented_train_30D",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "/home/data/sample_submission.csv",
    "/home/data/test",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

150720 3140


In [10]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [11]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [12]:
%%time
#from torch.utils.tensorboard import SummaryWriter

# TensorBoard SummaryWriter
#writer = SummaryWriter(log_dir='runs/exp1')

for epoch in range(EPOCHS):
    ret = train_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

    # TensorBoard에 기록합니다.
    #writer.add_scalar('Loss/train', ret['train_loss'], epoch)
    #writer.add_scalar('Accuracy/train', ret['train_acc'], epoch)
    #writer.add_scalar('F1_Score/train', ret['train_f1'], epoch)

#writer.close()

Loss: 0.0312: 100%|██████████| 2355/2355 [10:17<00:00,  3.82it/s]


train_loss: 0.2017
train_acc: 0.9315
train_f1: 0.9263
epoch: 0.0000



Loss: 0.0675: 100%|██████████| 2355/2355 [10:11<00:00,  3.85it/s]


train_loss: 0.0174
train_acc: 0.9942
train_f1: 0.9937
epoch: 1.0000



Loss: 0.0003: 100%|██████████| 2355/2355 [05:53<00:00,  6.66it/s]


...

train_loss: 0.0031
train_acc: 0.9990
train_f1: 0.9990
epoch: 7.0000



Loss: 0.0010: 100%|██████████| 2355/2355 [04:51<00:00,  8.07it/s]


train_loss: 0.0028
train_acc: 0.9992
train_f1: 0.9991
epoch: 8.0000



Loss: 0.0000: 100%|██████████| 2355/2355 [04:50<00:00,  8.11it/s]


train_loss: 0.0024
train_acc: 0.9993
train_f1: 0.9992
epoch: 9.0000

CPU times: user 1h 31s, sys: 5min 49s, total: 1h 6min 20s
Wall time: 1h 17s


In [ ]:
Loss: 0.0778: 100%|██████████| 1570/1570 [03:12<00:00,  8.15it/s]
train_loss: 0.2847
train_acc: 0.9040
train_f1: 0.8965
epoch: 0.0000

Loss: 0.0044: 100%|██████████| 1570/1570 [03:13<00:00,  8.11it/s]
train_loss: 0.0339
train_acc: 0.9884
train_f1: 0.9874
epoch: 1.0000

Loss: 0.0059: 100%|██████████| 1570/1570 [03:13<00:00,  8.12it/s]
train_loss: 0.0133
train_acc: 0.9958
train_f1: 0.9953
epoch: 2.0000

Loss: 0.0009: 100%|██████████| 1570/1570 [03:13<00:00,  8.12it/s]
train_loss: 0.0110
train_acc: 0.9963
train_f1: 0.9961
epoch: 3.0000

Loss: 0.0002: 100%|██████████| 1570/1570 [03:13<00:00,  8.11it/s]
train_loss: 0.0083
train_acc: 0.9974
train_f1: 0.9972
epoch: 4.0000

Loss: 0.0005: 100%|██████████| 1570/1570 [03:13<00:00,  8.11it/s]
train_loss: 0.0052
train_acc: 0.9984
train_f1: 0.9983
epoch: 5.0000

Loss: 0.0784:  12%|█▏        | 183/1570 [00:22<02:50,  8.13it/s]

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [13]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 50/50 [00:13<00:00,  3.64it/s]


In [14]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [15]:
sample_submission_df = pd.read_csv("/home/data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [16]:
pred_df.to_csv("/home/data/eff_b2_224_augmentation(30D,FlxBlxNo).csv", index=False)

In [17]:
# pred_df Macro F1 score 계산
from sklearn.metrics import f1_score
f1_score(sample_submission_df['target'], pred_df['target'], average='macro')

0.007110923363959586

In [18]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,12
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,4
4,00901f504008d884.jpg,2
